# 論文輪読
那須野薫(Kaoru Nasuno)/ 東京大学松尾研究室(Matsuo Lab., the University of Tokyo)

##メタ情報
タイトル：Learning Phrase Representations using RNN Encoder–Decoder for Statistical Machine Translation  
著者：Cho, K., Van Merriënboer, B., Gulcehre, C., Bahdanau, D., Bougares, F., Schwenk, H., & Bengio, Y.   
公開：2014  
リンク： [arXiv](http://arxiv.org/abs/1406.1078)  
被引用件数：84  

## Abstract
この論文では、2つのRNNからなるRNN Encoder-Decoderという新しいニューラルネットワークを提案する。  
ひとつのRNNがシンボルのシーケンスを固定長のベクトル表現にエンコードし、もうひとつがその表現をシンボルのシーケンスにデコードするというもの。  
提案モデルのエンコーダとデコーダはソースシーケンスをgivenとしたときのターゲットシーケンスの条件付き確率を最大化するようにひとつのネットワークとして学習させる。  
既存のlog-linearモデルの素性にRNN Encoder-Decoderにより算出されたフレーズペアの条件付き確率を利用することで、統計的機械翻訳システムの性能が向上することが実験的に示される。  
定性的には、提案モデルが言語フレーズの意味的・文法的に意味のある表現を学習することを示す。


##選択理由
#### Gated Recurrent Unitが気になっていた。
- 最近RNNをいじっている。
- LSTMよりマトリックス変数が少ないが精度が同程度？らしい。
- 数式の理解と実装が簡単に見える。

#### 可変長データを固定長に直し、さらに可変長にする方法を提案している。

#### 統計的機械翻訳に特別強い興味がある、というわけではない(むしろは薄いほう)。


## 目次
- RNN
- RNN Encoder-Decoder
- LSTM
- GRU

## RNN
<img src='files/RNN2.png' width="400px"/>
$$h_{<t>} = f(h_{<t-1>}, x_t)$$  

例えば、  
$$h_{<t>} = \sigma(W_xx_t + W_hh_{<t-1>})$$  
$W_h, W_x$：パラメタ





In [4]:
import theano
import theano.tensor as T
import numpy
rng = numpy.random.RandomState(1234)
x = T.fmatrix('x')
n_x = 10
n_hidden =10
Wh = theano.shared(
    rng.uniform(
        low=-numpy.sqrt(6. / (n_hidden + n_hidden)),
        high=numpy.sqrt(6. / (n_hidden + n_hidden)),
        size=(n_hidden, n_hidden),
    ).astype('float32'),
    name='Wh'
)

Wx = theano.shared(
    rng.uniform(
        low=-numpy.sqrt(6. / (n_x + n_hidden)),
        high=numpy.sqrt(6. / (n_x + n_hidden)),
        size=(n_x, n_hidden),
    ).astype('float32'),
    name='Wx'
)

def step(x, h_tm1, Wx, Wh):
    return T.nnet.sigmoid(T.dot(x, Wx) + T.dot(h_tm1, Wh))

h_t, updates = theano.scan(
    fn=step,
    sequences=x,
    outputs_info=numpy.zeros(n_hidden).astype('float32'),
    non_sequences=[Wx, Wh]
)

## RNN Encoder-Decoder
<img src='files/RNN_Encoder-Decoder.png' width="300px"/>
$$論文より引用$$
任意の長さのデータを固定長に変換して、さらに任意の長さに変換するRNN。  
終了記号(=$x_T$)を読み込んだあとのhidden stateが全体の入力のサマリー(=$c$)になっている。  
エンコーダ部分は普通のRNNと同じ。  
デコーダ部分は$y_t$も$h_{<t>}$も$y_{t-1}$の条件づけられる。  
$$h_{<t>}=f(h_{<t-1>}, y_{t-1}, c)$$  
$$y_{t}=f(h_{<t>}, y_{t-1}, c)$$  
学習では下記の$\theta$を求める。  
$$\max_{\theta}\frac{1}{N}\sum_{n=1}^Nlogp_{\theta}(y_n|x_n)$$



In [ ]:
y = T.fvector('y')
Wh = theano.shared(
    rng.uniform(
        low=-numpy.sqrt(6. / (n_hidden + n_hidden)),
        high=numpy.sqrt(6. / (n_hidden + n_hidden)),
        size=(n_hidden, n_hidden),
    ).astype('float32'),
    name='Wh'
)

##さいごに
間違い等あったら、ご連絡ください。